In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import re
import warnings
import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, log_loss
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

warnings.filterwarnings('ignore')

print("==================================================")
print("   HYPERPARAMETER TUNING ENGINE (ELO + ENSEMBLE)  ")
print("==================================================")

# ==========================================
# 1. LOAD & CLEAN (Same Feature Engineering)
# ==========================================
# We must recreate the exact features (ELO, Rolling, Rest) first
df = pd.read_csv("match_data.csv")

def extract_date(url):
    try:
        match = re.search(r'([A-Za-z]+-\d{1,2}-\d{4})', str(url))
        if match: return pd.to_datetime(match.group(1), format='%B-%d-%Y', errors='coerce')
    except: pass
    return pd.NaT

df['date'] = df['match_url'].apply(extract_date)
df = df.sort_values(by='date').reset_index(drop=True)

# Basic Numeric Cleaning & Imputation
stats_cols = ["xg", "possession", "shots_onTarget", "corners", "fouls", "team_points"]
for side in ['home', 'away']:
    p_col = f"{side}_team_possession"
    if p_col in df.columns:
        df[p_col] = pd.to_numeric(df[p_col].astype(str).str.rstrip('%'), errors='coerce').fillna(50) / 100.0
    
    # Calculate points if missing (3 for win logic)
    if 'team_points' not in df.columns and f'{side}_team_score' in df.columns:
        # (Logic handled in rolling block below)
        pass

    for s in stats_cols:
        col = f"{side}_{s}" if s not in ['xg', 'possession', 'team_points'] else f"{side}_team_{s}"
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            # Group median fill
            df[col] = df[col].fillna(df.groupby(f'{side}_team_name')[col].transform('median'))
            df[col] = df[col].fillna(df[col].mean()) # Fallback

# --- ELO CALCULATION ---
def calculate_elo(df):
    k_factor = 20
    home_advantage = 65 # Lowered slightly for modern football
    team_elos = {team: 1500 for team in set(df['home_team_name']).union(set(df['away_team_name']))}
    elo_h, elo_a = [], []
    
    for _, row in df.iterrows():
        h, a = row['home_team_name'], row['away_team_name']
        rh, ra = team_elos[h], team_elos[a]
        elo_h.append(rh); elo_a.append(ra)
        
        if row['home_team_score'] > row['away_team_score']: res = 1
        elif row['home_team_score'] == row['away_team_score']: res = 0.5
        else: res = 0
        
        dr = (rh + home_advantage) - ra
        e_h = 1 / (1 + 10 ** (-dr / 400))
        team_elos[h] = rh + k_factor * (res - e_h)
        team_elos[a] = ra + k_factor * ((1 - res) - (1 - e_h))
        
    return elo_h, elo_a

df['home_elo'], df['away_elo'] = calculate_elo(df)
df['diff_elo'] = (df['home_elo'] + 65) - df['away_elo']

# --- REST DAYS ---
def calc_rest(df):
    long_df = pd.concat([
        df[['date', 'home_team_name']].rename(columns={'home_team_name':'team'}),
        df[['date', 'away_team_name']].rename(columns={'away_team_name':'team'})
    ]).sort_values(['team', 'date'])
    long_df['rest'] = (long_df['date'] - long_df.groupby('team')['date'].shift(1)).dt.days.fillna(7).clip(upper=14)
    return dict(zip(zip(long_df['date'], long_df['team']), long_df['rest']))

rest_map = calc_rest(df)
df['diff_rest'] = df.apply(lambda x: rest_map.get((x['date'], x['home_team_name']),7), axis=1) - \
                  df.apply(lambda x: rest_map.get((x['date'], x['away_team_name']),7), axis=1)

# --- ROLLING STATS ---
# Recalculate points specifically for rolling
df['home_team_points'] = np.select([df['home_team_score']>df['away_team_score'], df['home_team_score']==df['away_team_score']], [3, 1], 0)
df['away_team_points'] = np.select([df['away_team_score']>df['home_team_score'], df['away_team_score']==df['home_team_score']], [3, 1], 0)

roll_feats = ['team_xg', 'team_possession', 'shots_onTarget', 'corners', 'team_points']
h_d = df[['date', 'match_url', 'home_team_name']].rename(columns={'home_team_name':'team'})
a_d = df[['date', 'match_url', 'away_team_name']].rename(columns={'away_team_name':'team'})

for f in roll_feats:
    c_h = f"home_{f}" if f in ['team_points', 'team_xg', 'team_possession'] else f"home_{f}"
    c_a = f"away_{f}" if f in ['team_points', 'team_xg', 'team_possession'] else f"away_{f}"
    if c_h in df.columns: h_d[f] = df[c_h]
    if c_a in df.columns: a_d[f] = df[c_a]

stacked = pd.concat([h_d, a_d]).sort_values(['team', 'date'])
for f in roll_feats:
    if f in stacked.columns:
        stacked[f'roll_{f}'] = stacked.groupby('team')[f].transform(lambda x: x.shift(1).rolling(5, min_periods=1).mean())

df = df.merge(stacked[['match_url', 'team'] + [f'roll_{f}' for f in roll_feats]], left_on=['match_url', 'home_team_name'], right_on=['match_url', 'team'], how='left').drop(columns=['team']).rename(columns={f'roll_{f}': f'home_roll_{f}' for f in roll_feats})
df = df.merge(stacked[['match_url', 'team'] + [f'roll_{f}' for f in roll_feats]], left_on=['match_url', 'away_team_name'], right_on=['match_url', 'team'], how='left').drop(columns=['team']).rename(columns={f'roll_{f}': f'away_roll_{f}' for f in roll_feats})

for f in roll_feats:
    df[f'diff_{f}'] = df[f'home_roll_{f}'] - df[f'away_roll_{f}']

df = df.fillna(0)

# ==========================================
# 2. PREPARE FOR TUNING
# ==========================================
conditions = [
    (df['home_team_score'] > df['away_team_score']),
    (df['home_team_score'] == df['away_team_score']),
    (df['home_team_score'] < df['away_team_score'])
]
df['match_outcome'] = np.select(conditions, [2, 1, 0])

features = [
    'diff_elo', 'home_elo', 'away_elo',
    'diff_rest', 'diff_team_points', 'diff_team_xg', 
    'diff_shots_onTarget', 'diff_corners',
    'home_roll_team_xg', 'away_roll_team_xg'
]

X = df[features]
y = df['match_outcome']

# Time Series Split (Crucial for validity)
# 4 splits = Test on last 20%, then last 40%, etc.
tscv = TimeSeriesSplit(n_splits=4)

print(f"Tuning on {len(X)} matches...")

# ==========================================
# 3. TUNE XGBOOST
# ==========================================
print("\n--- Tuning XGBoost ---")
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.03, 0.05],
    'max_depth': [3, 4],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8],
    'gamma': [1, 5] # Regularization
}

xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=3, tree_method='hist', random_state=42)
xgb_search = GridSearchCV(xgb_model, xgb_param_grid, cv=tscv, scoring='accuracy', n_jobs=-1, verbose=1)
xgb_search.fit(X, y)

print(f"✅ Best XGB Params: {xgb_search.best_params_}")
print(f"✅ Best XGB Score: {xgb_search.best_score_:.2%}")
best_xgb = xgb_search.best_estimator_

# ==========================================
# 4. TUNE RANDOM FOREST
# ==========================================
print("\n--- Tuning Random Forest ---")
rf_param_grid = {
    'n_estimators': [200, 400],
    'max_depth': [5, 8, 10], # Keep shallow to prevent overfitting noise
    'min_samples_leaf': [3, 5, 10], # Higher = more generalized
    'max_features': ['sqrt', 0.5]
}

rf_model = RandomForestClassifier(random_state=42)
rf_search = GridSearchCV(rf_model, rf_param_grid, cv=tscv, scoring='accuracy', n_jobs=-1, verbose=1)
rf_search.fit(X, y)

print(f"✅ Best RF Params: {rf_search.best_params_}")
print(f"✅ Best RF Score: {rf_search.best_score_:.2%}")
best_rf = rf_search.best_estimator_

# ==========================================
# 5. TUNE LOGISTIC REGRESSION
# ==========================================
print("\n--- Tuning Logistic Regression ---")
lr_param_grid = {
    'C': [0.01, 0.1, 1.0, 10.0], # Regularization strength
    'solver': ['lbfgs', 'newton-cg']
}

scaler = StandardScaler() # LR needs scaled data
X_scaled = scaler.fit_transform(X)

lr_model = LogisticRegression(multi_class='multinomial', max_iter=2000)
lr_search = GridSearchCV(lr_model, lr_param_grid, cv=tscv, scoring='accuracy', n_jobs=-1)
lr_search.fit(X_scaled, y)

print(f"✅ Best LR Params: {lr_search.best_params_}")
print(f"✅ Best LR Score: {lr_search.best_score_:.2%}")
best_lr = lr_search.best_estimator_

# ==========================================
# 6. BUILD FINAL ENSEMBLE
# ==========================================
print("\n--- Training Final Ensemble ---")

# We retrain the best models on the FULL dataset (minus the very last test chunk for final verification)
split = int(len(X) * 0.90)
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

# Note: LR needs scaling pipeline
from sklearn.pipeline import make_pipeline
pipe_lr = make_pipeline(StandardScaler(), best_lr)

ensemble = VotingClassifier(
    estimators=[
        ('xgb', best_xgb),
        ('rf', best_rf),
        ('lr', pipe_lr)
    ],
    voting='soft',
    # Weight based on validation scores. 
    # Usually XGB > LR > RF for this data size.
    weights=[3, 1, 2] 
)

ensemble.fit(X_train, y_train)

# ==========================================
# 7. FINAL EVALUATION
# ==========================================
preds = ensemble.predict(X_test)
probs = ensemble.predict_proba(X_test)
acc = accuracy_score(y_test, preds)

print("==================================================")
print(f"   TUNED ENSEMBLE ACCURACY: {acc:.2%}   ")
print("==================================================")

# Value Bets Check
results = pd.DataFrame({
    'Home': df.iloc[split:]['home_team_name'],
    'Away': df.iloc[split:]['away_team_name'],
    'Actual': y_test.values,
    'Pred': preds,
    'Conf': np.max(probs, axis=1)
})

for t in [0.45, 0.50, 0.60]:
    sub = results[results['Conf'] > t]
    if len(sub) > 0:
        print(f"Threshold > {t:.2f}: {len(sub)} matches | Accuracy: {accuracy_score(sub['Actual'], sub['Pred']):.2%}")

# Save Everything
joblib.dump({
    'model': ensemble, 
    'features': features, 
    'scaler': scaler,
    'elo_dict': dict(zip(df['home_team_name'], df['home_elo'])) # Save latest ELOs
}, 'tuned_ensemble_model.pkl')